# TODO

- nacitat a poupratat dataset v inom ntb, ulozit .csv
- v tomto vytvorit univerzalny prompt s premennou {domain name} a {input to classify}
- v dalsom spravit neuronku na klasifikaciu zrejme LSTM + CNN head (toto mozno) 
- SVM+TF-IDF alebo nejaky iny tradicny sposob klasifikaciu 

In [8]:
import os
import dotenv
import dspy
import json
import requests

In [3]:
dotenv.load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")
proxy_url = os.getenv("PROXY_URL")
model = "gpt-4o-mini"

In [11]:
def get_models() -> list:
    url = f'{proxy_url}models'
    headers = {"Content-Type": "application/json", "Authorization": f"Bearer {api_key}"}

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        models = response.json()
        models = [model["id"] for model in models["data"]]

        return models

    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")

        return None

# Example usage
available_models = get_models()

if available_models:
    print("Available models:", available_models)

else:
    print("Failed to retrieve models")

Available models: ['gpt-4o-mini']


In [4]:
lm = dspy.LM(
    api_key=api_key,
    model=model,
    api_base=proxy_url,
    temperature=0.2,
)

dspy.configure(lm=lm)

In [5]:
class ClasificationSignature(dspy.Signature):
    """Classify if the input text is in law domain or not. If you are not sure about the domain classify as neutral."""

    prompt = dspy.InputField(desc="The input text to classify.")

    explanation = dspy.OutputField(desc="Reasoning behind the classification.")
    answer = dspy.OutputField(desc="Yes if the input text is domain specific, No otherwise. Neutral if not sure.")


class ClassificationSimple(dspy.Module):
    def __init__(self) -> None:
        super().__init__()

        self.prog = dspy.ChainOfThought(ClasificationSignature)
    
    def forward(self, prompt: str) -> ClasificationSignature:
        prediction = self.prog(prompt=prompt)
        return prediction

In [6]:
test_query = 'In order to be sued in a particular jurisdiction, say New York, a company must have a minimal business presence in the jurisdiction. What constitutes such a presence? Suppose the company engaged a New York-based Plaintiff, and its representatives signed the contract with the Plaintiff in New York City. Does this satisfy the minimum presence rule? Suppose, instead, the plaintiff and contract signing were in New Jersey, but the company hired a law firm with offices in New York City. Does this qualify?'
test_answer = 'Historically, the threshold for "doing business" for purposes of jurisdiction was been very low. Any regular office or building used for the transaction of business would quality as would any work "directly" particularly at a state (e.g. if you did architectural work for a factory in Mississippi from your offices in Alaska, you would be "doing business" in Mississippi). Also, participation in a trade show would normally constitute "doing business" for a transaction arising out of the trade show. A string of recent U.S. Supreme Court cases have tended to interpret this phrase more narrowly, but so long as a lawsuit has a nexus with the instance of "doing business" in a state that is in question in the lawsuit, the definition isnt that much different for most purposes. A subsidiary or affiliate companys operations in a state will ordinarily not constitute doing business in a state on behalf of the parent company under recent U.S. Supreme Court precedents. Suppose the company engaged a New York-based Plaintiff, and its representatives signed the contract with the Plaintiff in New York City. Does this satisfy the minimum presence rule? Yes. Suppose, instead, the plaintiff and contract signing were in New Jersey, but the company hired a law firm with offices in New York City. Does this qualify? This could depend upon the subject matter of the contract and where the negotiations took place.'

input = dspy.Example(prompt=test_query, explanation=test_answer, answer="Yes").with_inputs("prompt")
output = ClassificationSimple()(input.prompt)

NameError: name 'output' is not defined

In [ ]:
print(output.answer)